# Custom Training with YOLOv5

In this tutorial, we assemble a dataset and train a custom YOLOv5 model to recognize the objects in our dataset. To do so we will take the following steps:

* Gather a dataset of images and label our dataset
* Export our dataset to YOLOv5
* Train YOLOv5 to recognize the objects in our dataset
* Evaluate our YOLOv5 model's performance
* Run test inference to view our model at work



![](https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/615627e5824c9c6195abfda9_computer-vision-cycle.png)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 1: Install Requirements

In [6]:
!cp -r /content/yolov5/ /content/drive/MyDrive/ProjectTests/

In [ ]:
!rm -r /content/yolov5/data/PPE-Dectection-1

In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 12078, done.
remote: Total 12078 (delta 0), reused 0 (delta 0), pack-reused 12078
Receiving objects: 100% (12078/12078), 11.88 MiB | 33.62 MiB/s, done.
Resolving deltas: 100% (8382/8382), done.
/content/yolov5
     |████████████████████████████████| 596 kB 30.6 MB/s 
     |████████████████████████████████| 145 kB 30.0 MB/s 
     |████████████████████████████████| 178 kB 61.0 MB/s 
     |████████████████████████████████| 1.1 MB 48.6 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 138 kB 60.4 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datas

# Step 2: Assemble Our Dataset

In order to train our custom model, we need to assemble a dataset of representative images with bounding box annotations around the objects that we want to detect. And we need our dataset to be in YOLOv5 format.

In Roboflow, you can choose between two paths:

* Convert an existing dataset to YOLOv5 format. Roboflow supports over [30 formats object detection formats](https://roboflow.com/formats) for conversion.
* Upload raw images and annotate them in Roboflow with [Roboflow Annotate](https://docs.roboflow.com/annotate).

# Annotate

![](https://roboflow-darknet.s3.us-east-2.amazonaws.com/roboflow-annotate.gif)

# Version

![](https://roboflow-darknet.s3.us-east-2.amazonaws.com/robolfow-preprocessing.png)


In [3]:
!pip install roboflow wandb
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 14.8 MB/s 
     |████████████████████████████████| 145 kB 73.1 MB/s 
     |████████████████████████████████| 181 kB 68.0 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=7284efc9b2b8c0795e519ad7daa1c64ead6ccba41a98ed8596d4b2571c12a93a
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [4]:
os.environ["DATASET_DIRECTORY"] = "/content/yolov5/data"

In [5]:
from roboflow import Roboflow
rf = Roboflow(api_key="tUautLA1Lc5HyBkpJbsP")
project = rf.workspace("sorokinevgeni-gmail-com").project("ppe-dectection")
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/yolov5/data/PPE-Dectection-1 in yolov5pytorch:: 100%|██████████| 45014/45014 [00:34<00:00, 1322.10it/s]


# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

# Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

In [ ]:
!python train.py --img 416 --batch-size -1 --epochs 50 --data {dataset.location}/data.yaml --weights yolov5n.pt --cache
!python train.py --img 416 --batch-size -1 --epochs 50 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache
!python train.py --img 416 --batch-size -1 --epochs 50 --data {dataset.location}/data.yaml --weights yolov5m.pt --cache
!python train.py --img 416 --batch-size -1 --epochs 50 --data {dataset.location}/data.yaml --weights yolov5l.pt --cache
!python train.py --img 416 --batch-size -1 --epochs 50 --data {dataset.location}/data.yaml --weights yolov5x.pt --cache

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 2
wandb: You chose 'Use an existing W&B account'
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
train: weights=yolov5n.pt, cfg=, data=/content/yolov5/data/PPE-Detection-4/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=-1, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_da

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs" 
%load_ext tensorboard
%tensorboard --logdir runs

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
!python detect.py --weights /content/yolov5/weights/5KAUTOv5x.pt --img 416 --conf 0.1 --source /content/PPEnew.mp4

detect: weights=['/content/yolov5/weights/5KAUTOv5x.pt'], source=/content/PPEnew.mp4, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-208-g15057d2 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 444 layers, 86213788 parameters, 0 gradients, 204.1 GFLOPs
video 1/1 (1/4310) /content/PPEnew.mp4: 256x416 1 Hardhat, Done. (0.036s)
video 1/1 (2/4310) /content/PPEnew.mp4: 256x416 1 Hardhat, Done. (0.033s)
video 1/1 (3/4310) /content/PPEnew.mp4: 256x416 1 Hardhat, Done. (0.033s)
video 1/1 (4/4310) /content/PPEnew.mp4: 256x416 1 Hardhat, Done. (0.033s)
video 1/1 (5/4310) /content/PPEnew.mp4: 256x416 1 Hardhat

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

Output hidden; open in https://colab.research.google.com to view.

# Conclusion and Next Steps

Congratulations! You've trained a custom YOLOv5 model to recognize your custom objects.

To improve you model's performance, we recommend first interating on your datasets coverage and quality. See this guide for [model performance improvement](https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results).

To deploy your model to an application, see this guide on [exporting your model to deployment destinations](https://github.com/ultralytics/yolov5/issues/251).

Once your model is in production, you will want to continually iterate and improve on your dataset and model via [active learning](https://blog.roboflow.com/what-is-active-learning/).

In [ ]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')

FileNotFoundError: ignored